## SaaS Interpretation of the Dataset

This dataset represents a subscription-based SaaS product. Customers subscribe to pricing plans, generate recurring revenue, and may churn over time. The data allows analysis of activation, retention, churn, monetization, and lifetime value (LTV).

Key assumptions:
- Each customer represents a unique SaaS user or account
- Subscription plans represent pricing tiers
- Revenue records represent recurring subscription payments
- Churn indicates subscription cancellation


## Core SaaS Metrics

- Activation Rate: Percentage of users who remain subscribed beyond an early tenure threshold
- Retention Rate: Percentage of users retained over time
- Churn Rate: Percentage of users who cancel subscriptions
- MRR (Monthly Recurring Revenue): Total recurring revenue per month
- ARPU (Average Revenue Per User)
- LTV (Customer Lifetime Value)


## Key Product Questions

1. When do customers churn most frequently?
2. Which subscription plans retain users the longest?
3. Which plans and customers generate the most revenue?
4. How does early customer behavior relate to long-term value?
5. What product or pricing changes could improve retention or revenue?


## Key Product Questions

1. When do customers churn most frequently?
2. Which subscription plans retain users the longest?
3. Which plans and customers generate the most revenue?
4. How does early customer behavior relate to long-term value?
5. What product or pricing changes could improve retention or revenue?

In [26]:
import pandas as pd

customers = pd.read_csv("../data/raw/customers.csv")
subscriptions = pd.read_csv("../data/raw/subscriptions.csv")
revenue = pd.read_csv("../data/raw/revenue.csv")

In [27]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       1000 non-null   int64 
 1   signup_date       1000 non-null   object
 2   plan_type         1000 non-null   object
 3   monthly_fee       1000 non-null   int64 
 4   acquisition_cost  1000 non-null   int64 
 5   churn_date        168 non-null    object
dtypes: int64(3), object(3)
memory usage: 47.0+ KB


In [28]:
customers.head()

,customer_id,signup_date,plan_type,monthly_fee,acquisition_cost,churn_date
0,1001,2024-11-07,Basic,50,30,NaN
1,1002,2024-06-06,Basic,50,30,NaN
2,1003,2024-12-31,Basic,50,30,NaN
3,1004,2024-11-21,Pro,200,100,NaN
4,1005,2024-08-16,Pro,200,100,NaN


In [29]:
subscriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   subscription_id  988 non-null    object
 1   customer_id      988 non-null    int64 
 2   month            988 non-null    object
 3   monthly_fee      988 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 31.0+ KB


In [30]:
subscriptions.head()

,subscription_id,customer_id,month,monthly_fee
0,S-1020-202410,1020,2024-10,200
1,S-1020-202411,1020,2024-11,200
2,S-1020-202412,1020,2024-12,200
3,S-1020-202501,1020,2025-01,200
4,S-1020-202502,1020,2025-02,200


In [31]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   subscription_id  988 non-null    object
 1   customer_id      988 non-null    int64 
 2   month            988 non-null    object
 3   monthly_fee      988 non-null    int64 
 4   revenue_type     988 non-null    object
 5   amount           988 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 46.4+ KB


In [32]:
revenue.head()

,subscription_id,customer_id,month,monthly_fee,revenue_type,amount
0,S-1020-202410,1020,2024-10,200,MRR,200
1,S-1020-202411,1020,2024-11,200,MRR,200
2,S-1020-202412,1020,2024-12,200,MRR,200
3,S-1020-202501,1020,2025-01,200,MRR,200
4,S-1020-202502,1020,2025-02,200,MRR,200


## Analytical Grain
All core metrics in this project will be analyzed at the customer-month level to support retention and revenue analysis.


In [33]:
customers['signup_date'] = pd.to_datetime(customers['signup_date'])
customers['churn_date'] = pd.to_datetime(customers['churn_date'])

subscriptions['month'] = pd.to_datetime(subscriptions['month'])
revenue['month'] = pd.to_datetime(revenue['month'])

In [34]:
#First subscription month per cusomer
first_subscription =(
    subscriptions
    .groupby('customer_id')['month']
    .min()
    .reset_index(name ='first_subscription_month')
)

In [35]:
first_subscription.head()

,customer_id,first_subscription_month
0,1020,2024-10-01
1,1021,2024-04-01
2,1023,2024-12-01
3,1026,2025-03-01
4,1031,2024-04-01


In [36]:
activation = customers.merge(
    first_subscription,
    on='customer_id',
    how='left'
)
activation['days_to_activate']=(
    activation['first_subscription_month'] - activation['signup_date']
).dt.days

In [37]:
activation[['days_to_activate']].describe()


,days_to_activate
count,168.000000
mean,-15.375000
std,8.877535
min,-30.000000
25%,-23.000000
50%,-15.000000
75%,-8.000000
max,0.000000


In [24]:
activation_rate = activation['first_subscription_month'].notna().mean()
activation_rate


np.float64(0.168)

In [41]:
activation['signup_month'] = activation['signup_date'].dt.to_period('M')
activation['first_subscription_month'] = activation['first_subscription_month'].dt.to_period('M')


In [43]:
activation['months_to_activate'] = (
    activation['first_subscription_month'] - activation['signup_month']
)

activation['months_to_activate'] = activation['months_to_activate'].apply(
    lambda x: x.n if pd.notnull(x) else None
)



In [44]:
activation['months_to_activate'].describe()


count    168.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: months_to_activate, dtype: float64

In [45]:
activation['months_to_activate'].value_counts().sort_index()


months_to_activate
0.0    168
Name: count, dtype: int64

## Activation Insight
All activated customers converted in the same month as signup, indicating that activation is immediate rather than gradual. Customers who fail to activate in their signup month rarely convert later, making early onboarding and first-month experience critical for long-term retention and revenue.
